Maintaining Data Privacy in Association Rule Mining using MASK algorithm 

By

Kozy-Korpesh Tolep, matricola: 5302354  

Salah Ismail , matricola: 5239380

Amir Mashmool, matricola: 5245307

## Background

This paper discusses the challenge of obtaining accurate input data for data mining services while addressing privacy concerns. It explores whether users can be motivated to provide correct information by guaranteeing privacy protection during the mining process. proposing a scheme that distorts user data probabilistically, ensuring a high level of privacy while maintaining accurate mining results. It is this distorted information that is eventually supplied to the data miner, along with a description of the distortion procedure. The performance of the scheme is validated using real and synthetic datasets. 


## Dataset 

The paper assumes a database model where each customer's data is represented as a tuple consisting of a fixed-length sequence of 1's and 0's. This model is commonly used for market-basket databases, where columns represent items sold by a supermarket, and each row represents a customer's purchases (1 indicating a purchase and 0 indicating no purchase).

The assumption is that the number of 1's (purchases) in the database is significantly smaller than the number of 0's (non-purchases) ==> In short, the database is modeled as a large disk-resident two-dimensional sparse boolean matrix.

## Mining Objectives

The mining objective is to efficiently discover all frequent itemsets in the database, which correspond to statistically significant and strong association rules. These rules have a support factor indicating their frequency and a confidence factor representing their strength. The goal is to find interesting rules that surpass user-defined thresholds for support and confidence. A rule is said to be “interesting” if its support and confidence are greater than user-defined thresh-olds $sup_{min}$ and $con_{min}$, respectively.

example of a association rule:

Transaction 1: {Bread, Milk}
Transaction 2: {Bread, Milk, Diapers}

Using these measures, we can discover association rules:
Bread → Milk (Support: 40%, Confidence: 66.6%)
   This rule indicates that if a customer buys bread, they are likely to buy milk as well.
Bread, Milk → Diapers (Support: 40%, Confidence: 50%)
    This rule states that if a customer buys both bread and milk, they are likely to buy diapers as well.



## The privacy metric

As stated in the paper the metric is “with what probability can a given 1 or 0 in the true matrix be reconstructed”.
It measures the probability of reconstructing distorted user data. It focuses on individual entries within customer tuples. For many applications, customers may prioritize more privacy for their 1's (purchased actions) compared to their 0's (non-purchased options).

## Quantifying MASK’s Privacy

In this section, we present the distortion procedure used by the MASK scheme and quantify the privacy provided by the procedure, as per the above privacy metric.

### Distortion Procedure

A customer tuple can be considered to be a random vector $ X = \{X_i\}$ , such that $X_i = 0 \ or \ 1$ .
We generate the distorted vector from this customer tuple by computing $Y = distort(X)$ where $Y_i=X_i \ XOR \ \overline r_i$ and $\overline r_i$ is the complement of $r_i$, a random variable with a density function $f(r) = bernoulli(p) \ (0 \leq p \leq 1)$. That is, $r_i$ takes a value 1 with probability $p$ and 0 with probability $1 - p$.

The net effect of the above computation is that the identity of the $i^{th}$ element in $X$ is kept the same with probability $p$ and is flipped with probability $(1 — p)$. All the customer tuples are distorted in this fashion and make up the database supplied to the miner in effect, the miner receives a $probabilistic function$ of the true customer database.

### Reconstruction Probability of a 1
${R}_1(p)$ is the probability with which a ‘1’ can be reconstructed from the distorted entry as follow:

$\mathcal{R}_1(p)=\frac{s_0 \times p^2}{s_0 \times p+\left(1-s_0\right) \times(1-p)}+\frac{s_0 \times(1-p)^2}{s_0 \times(1-p)+\left(1-s_0\right) \times p}$

while $s_0$ is the average support of an item in the database. 

### The General Reconstruction Equation
the relationship between $p$ and the reconstruction probability for the general case where the customer may wish to protect both his $1’s$ and $0’s$, but his concern to keep the $1’s$ private is more than that for the $0’s$.

${R}_0(p)$ is the probability with which a ‘0’ can be reconstructed from the distorted entry as follow:

$\mathcal{R}_0(\mathrm{p})=\frac{\left(1-s_0\right) \times p^2}{\left(1-s_0\right) \times p+s_0 \times(1-p)}+\frac{\left(1-s_0\right) \times(1-p)^2}{s_0 \times p+\left(1-s_0\right) \times(1-p)}$

Our aim is to minimize a weighted average of ${R}_1(p)$ and ${R}_0(p)$. This corresponds to minimizing the probability of reconstruction of both $1’s$ and $0’s$. The $\textbf {total reconstruction probability}$, ${R}(p)$, is then given as:

$\mathcal{R}(p)=a \mathcal{R}_1(p)+(1-a) \mathcal{R}_0(p)$

where $a$ is the weight given to $1’s$ over $0’s$.


## Privacy Measure
Armed with the ability to compute the total reconstruction probability, we now simply define $\textbf {user privacy}$ ${P}(p)$ as the following percentage:

$\mathcal{P}(p)=(1-\mathcal{R}(p)) * 100$



distortion_probability = the distortion probability value given to distortion function

distort_dataset() = a function used to distort the dataset based on distortion_probability

T = the original true matrix dataset

D = the distorted matrix dataset obtained with a distortion probability

In [ ]:
import numpy as np
import random 

def distort_dataset(T, distortion_probability):
    mask = np.random.random(T.shape) > (1 - distortion_probability)
    D = (T + mask) % 2
    return D


$s_0$ is the average support of an item in the database.

$a$ is the weight given to $1’s$ over $0’s$.

${P}(p)$ is user privacy metric

###### Computing ${R}_1(p)$ and ${R}_0(p)$
###### Computing the total reconstruction probability ${R}(p)$
###### Computing the privacy metric ${P}(p)$

In [3]:
def user_privacy(s_0, a, p):
    r1_p = ((s_0 * (p ** 2)) / ((s_0 * p) + ((1 - s_0) * (1 - p)))) + ((s_0 * ((1 - p) ** 2)) / ((s_0 * (1 - p)) + ((1 - s_0) * p)))
    r0_p = (((1 - s_0) * (p ** 2)) / (((1 - s_0) * p) + (s_0 * (1 - p)))) + ((s_0 * ((1 - p) ** 2)) / ((s_0 * p) + ((1 - s_0) * (1 - p))))
    
    r_p = (a * r1_p) + ((1 - a) * r0_p)
    
    p_p = (1 - r_p) * 100
    
    return p_p


## Mining the Distorted Database (MASK Algorithm)

MASK’s technique is for estimating the true (accurate) supports of itemsets from a distorted database. 
We first show how to estimate the supports of $1-itemsets$ (i.e. singletons) and then present the general $n-itemset$ support estimation procedure.

It is important to keep in mind that the miner is provided with both the distorted matrix as well as the distortion procedure, that is, it knows the value of $p$ that was used in distorting the true matrix.

### Estimating $1-itemsets$ Supports
We denote the original true matrix by $T$ and the distorted matrix, obtained with a distortion probability of $p$, as $D$.

Now consider a random individual item $i$. Let $c_1^T$ and $c_0^T$ represent the number of $1’s$ and $0’s$, respectively, in the $i4 column of $T$, while $c_1^D$ and $c_0^D$ represent the number of $1’s$ and $0’s$, respectively, in the $i$ column of $D$. With this notation, we estimate the support of $i$ in $T$ using the following equation:

$\mathbf{C}^T=\mathbf{M}^{-1} \mathbf{C}^{\mathrm{D}}$

Where

$M=\left[\begin{array}{cc}p & 1-p \\ 1-p & p\end{array}\right] \  C^D=\left[\begin{array}{c}c_1^D \\ c_0^D\end{array}\right] C^T=\left[\begin{array}{c}c_1^T \\ c_0^T\end{array}\right]$

The M matrix in the above equation incorporates the observation that by our method of distortion, if a column had $n \ 1’s$ in $T$, these $1’s$ will generate approximately $pn \ 1’s$ and $(1 — p)n \ 0’s$ for the same column in $D$. Similarly for the $0’s$ of this column in $T$. Therefore, given $c_1^D$ and $c_0^D$, it is possible to estimate the value of $c_1^T$, that is, the true support of item $i$.


### Estimating $n-itemsets$ Supports
To compute the support for an arbitrary n-itemset. For this general case, we define the matrices as:

$C^D=\left[\begin{array}{c}c_{2^n-1}^D \\ \cdot \\ \cdot \\ \cdot \\ c_1^D \\ c_0^D\end{array}\right] \quad \ C^T=\left[\begin{array}{c}c_{2^n-1}^T \\ \cdot \\ \cdot \\ \cdot \\ c_1^T \\ c_0^T\end{array}\right]$

Here $c_k^T$ should be interpreted as the count of the tuples in $T$ that have the binary form of $k$ (in $n$ digits) for the given itemset (that is, for a 2 -itemset, $c_2^T$ refers to the count of 10's in the columns of $T$ corresponding to that itemset, $c_3^T$ to the count of 11 's, and so on). Similarly, $c_k^D$ is defined for the distorted matrix $D$. Finally, the matrix $\mathbf{M}$ is defined as:

$m_{i, j}=$ The probability that a tuple of the form corresponding to $c_j^T$ in $T$ goes to a tuple of the form corresponding to $c_i^D$ in $D$.

For example, $m_{1, 2}=$ for a $2-itemset$ is the probability that a $10$ tuple distorts to a $01$ tuple. Accordingly, $m_{1, 2}=$ $(1-p)(1-p)$. The basis for this formulation lies in the fact that in our distortion procedure, the component columns of an $n-itemset$ are distorted $idependently$. Therefore, we can use the product of the probability terms.

### The Full Mining Process
The above equations help us to estimate the value of $c_{2^n-1}^T$ for an $n-itemset$ by using the values of $c_i^D$, $0 \leq i \leq 2^n - 1$. But, first we need to compute the $c_i^D$ values themselves. For this purpose, after the modifications described below, we have currently implemented our system based on the classical Apriori algorithm.

The Apriori algorithm is a multi-pass algorithm that discovers frequent itemsets by incrementally increasing the length of itemsets. It uses the AprioriGen algorithm to generate candidate itemsets for each pass based on the frequent itemsets found in the previous pass. This approach helps efficiently search for frequent itemsets in large datasets by focusing on subsets that are likely to be frequent.

In our approach the "MASK", unlike the Apriori algorithm, we consider and keep track of all possible combinations when counting 2-itemsets. Apriori only counts the occurrences of '11' in the tuples for each candidate 2-itemset, indicating both items appearing together. However, our approach requires tracking the counts for all combinations: '00', '01', '10', and '11'. This difference in counting allows us to capture more detailed information about itemset co-occurrences in the data.

## MASK Mining Optimizations

### Linear Number of Counters
### Reducing Amount of Counting

In [ ]:
# the MASK() here

## Performance Framework
Due to the probabilistic nature of MASK, the reconstructed support values are not expected to match the actual supports exactly. This can lead to errors where the reported support values are either larger or smaller than the actual supports.

Errors in support estimation can have a significant impact, the probabilistic evaluation of supports may incorrectly classify "border-line" itemsets as either frequent or rare. This results in both false positives (itemsets wrongly reported as frequent) and false negatives (itemsets wrongly reported as rare).

We evaluate the mining process under two conditions. The first condition uses the $sup_{min}$ value provided by the user, while the second condition uses a slightly lower $sup_{min}$ value.

We prioritize coverage over precision in this evaluation. Specifically, they consider a 10% reduction in the $sup_{min}$ value (r = 10%).

To quantify the errors made, We compare the mining outputs obtained from MASK with those derived from Apriori running on the true (undistorted) database. We compare the results using both the $sup_{min}$ value provided by the user and the lowered $sup_{min}$ value. This allows to evaluate the impact of the lowered $sup_{min}$ value on the mining process and assess the differences between MASK and Apriori.

### Error Metrics

1.Support Error  $\rho$.

This metric reflects the (percentage) average relative error in the reconstructed support values for those itemsets that are correctly  identified to be frequent. Denoting the reconstructed support by $rec_sup$ and the actual support by $act_sup$, the support error is computed over all frequent itemsets as

$\rho=\frac{1}{|f|} \Sigma_f \frac{\left|r e c_{-} s u p_f-a c t_{-} s u p_f\right|}{a c t_{\_} s u p_f} * 100$

2.Identity Error.

This metric reflects the percentage error in identifying frequent itemsets and has two components:$\sigma^{+}$, indicating the percentage of false positives, and $\sigma^{-} indicating the percentage of false negatives. Denoting the reconstructed set of frequent itemsets with $R$ and the correct set of frequent itemsets with $F$, these metrics are computed as:

$\sigma^{+}=\frac{|R-F|}{|F|} * 100 \quad \sigma^{-}=\frac{|F-R|}{|F|} * 100$

In [ ]:
import numpy as np

def calculate_support_error(mask_f, mask_f_s, apriori_f_s):
    """
    Calculates the support error metric.

    Args:
    mask_f (list): List of frequent itemsets obtained from MASK.
    mask_f_s (list): List of estimated supports of frequent itemsets from MASK.
    apriori_f_s (list): List of actual supports of frequent itemsets from Apriori.

    Returns:
    float: Support error.
    """
    total_itemsets = len(mask_f)
    support_error = sum([abs(rec_sup - act_sup) / act_sup for rec_sup, act_sup in zip(mask_f_s, apriori_f_s)]) / total_itemsets 
    return support_error

def calculate_identity_error(mask_f, apriori_f):
    """
    Calculates the identity error metrics.

    Args:
    mask_f (list): List of frequent itemsets obtained from MASK.
    apriori_f (list): List of frequent itemsets obtained from Apriori.

    Returns:
    tuple: Identity error (false positives, false negatives).
    """
    reconstructed_itemsets = set(mask_f)
    actual_itemsets = set(apriori_f)
    false_positives = len(reconstructed_itemsets - actual_itemsets) / len(actual_itemsets)
    false_negatives = len(actual_itemsets - reconstructed_itemsets) / len(actual_itemsets)
    return false_positives, false_negatives

def support_identity_errors(mask_f, mask_f_s, apriori_f, apriori_f_s):
    nbr_m_f = np.size(mask_f)
    levels = len(mask_f[nbr_m_f-1])

    L = []
    S_e = []
    F_p = []
    F_n = []

    for l in range(levels):
        level = l+1
        level_mask_f = []
        level_mask_f_s = []
        for i in range(np.size(mask_f)):
            if len(mask_f[i]) == level:
                level_mask_f.append(mask_f[i])
                level_mask_f_s.append(mask_f_s[i])

        support_error = calculate_support_error(level_mask_f, level_mask_f_s, apriori_f_s)

        false_positive, false_negative = calculate_identity_error(level_mask_f, apriori_f)

        L.append(level)
        S_e.append(support_error)
        F_p.append(false_positive)
        F_n.append(false_negative)

    return L, S_e, F_p, F_n
